# Case. Impact of Agent satisfaction sensibility on under-assignment

Situation:

    More agent availability on week days as weekends, 40% agents work on weekends, 60% not.
    NOT INCLUDED: Shift requirements higher on Friday 3rd Shift (3S), Saturday 1S and 3S, Sunday 3S
    Beta = 0.8

Task:

    Evaluate Shift Coverage over Agent satisfaction penalty for under-assignment
    Evaluate Agent Satisfaction over Agent satisfaction penalty for under-assignment using default function and shift stability function

In [1]:
import abm_scheduling
from abm_scheduling import Schedule as Schedule
from abm_scheduling import Nurse as Nurse

In [2]:
import time
from datetime import datetime

import abm_scheduling.Log
from abm_scheduling.Log import Log as Log

log = Log()

## Randomized. Define situation

In [3]:
p_to_accept_negative_change = .001
num_nurses_per_shift = 4
beta = 0.8

In [4]:
# 25 nurses, only 10 can work on weekends
nurses = []
for n in range(10):
    nurse = Nurse(id_name=n)
    nurse.generate_shift_preferences(degree_of_agent_availability=0.7, works_weekends=True)
    nurses.append(nurse)

In [5]:
for n in range(15):
    nurse = Nurse(id_name=(10+n))
    nurse.generate_shift_preferences(degree_of_agent_availability=0.7, works_weekends=False)
    nurses.append(nurse)

In [6]:
schedule = Schedule(num_nurses_needed=num_nurses_per_shift, is_random=True)
model = abm_scheduling.NSP_AB_Model()
schedule.print_schedule(schedule_name="Intial Situation")

Week's Schedule Intial Situation
+---------+----------+----------+----------+----------+----------+----------+----------+
|         | Mo       | Tu       | We       | Th       | Fr       | Sa       | So       |
+---------+----------+----------+----------+----------+----------+----------+----------+
| shift 1 | need: 4  | need: 5  | need: 6  | need: 6  | need: 5  | need: 4  | need: 3  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 2 | need: 7  | need: 4  | need: 5  | need: 5  | need: 4  | need: 5  | need: 6  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 3 | need: 4  | need: 4  | need: 6  | need: 6  | need: 4  | need: 5  | need: 6  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
+---------+----------+----------+----------+----------+----------+----------+----------+


In [7]:
model.show_hypothetical_max_schedule(schedule=schedule, nurses=nurses, print_detail_schedule = False)

Crude hypothetical shift coverage: 1.0
Shift Coverage Hypothetical Maximum
+---------+------------+------------+------------+------------+------------+-----------+-----------+
|         | Mo         | Tu         | We         | Th         | Fr         | Sa        | So        |
+---------+------------+------------+------------+------------+------------+-----------+-----------+
| shift 1 | need: 4    | need: 5    | need: 6    | need: 6    | need: 5    | need: 4   | need: 3   |
|         | nurses: 17 | nurses: 15 | nurses: 20 | nurses: 20 | nurses: 17 | nurses: 5 | nurses: 6 |
|         | (4.25)     | (3)        | (3.33)     | (3.33)     | (3.4)      | (1.25)    | (2)       |
| shift 2 | need: 7    | need: 4    | need: 5    | need: 5    | need: 4    | need: 5   | need: 6   |
|         | nurses: 16 | nurses: 15 | nurses: 17 | nurses: 21 | nurses: 20 | nurses: 7 | nurses: 7 |
|         | (2.29)     | (3.75)     | (3.4)      | (4.2)      | (5)        | (1.4)     | (1.17)    |
| shift 3 | need

In [ ]:
model.get_total_agent_satisfaction(nurses)

In [ ]:
# Default utility function (no agent satisfaction)
start_time = datetime.today()
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.8, 
                         p_to_accept_negative_change=.001, utility_function_parameters = None)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

In [ ]:
model.print_nurse_productivity(results.nurses)

In [ ]:
# Agent satisfaction utility function
utility_function_parameters = abm_scheduling.Utility_Function_Parameters()
utility_function_parameters.utility_function = 'agent_satisfaction'
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.8, 
                         p_to_accept_negative_change=.001, utility_function_parameters = utility_function_parameters)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

In [ ]:
model.print_nurse_productivity(results.nurses)

## Targeted search. Define situation

In [ ]:
#%% Initializations
p_to_accept_negative_change = .001

# Situation definition
matrix_nurses_needed = [5,6,3, 5,6,3, 5,6,3, 5,6,3, 5,6,5, 6,6,5, 6,6,3]
#type 1 only first shift work week days
matrix_nurse_availability_type1 = ['x','','', 'x','','',  'x','','', 'x','','', 'x','','', 'x','','', '','','x']
#type 2 1s+2s work week days
matrix_nurse_availability_type2 = ['x','x','', 'x','x','',  'x','x','', 'x','x','', 'x','x','', '','','', '','','']
#type 3 1s+2s+3s work week days
matrix_nurse_availability_type3 = ['x','x','x', 'x','x','x',  'x','x','x', 'x','x','x', 'x','x','', 'x','x','', 'x','x','']
#type 4 "springers"
matrix_nurse_availability_type4 = ['x','x','x', 'x','x','x',  'x','x','x', 'x','x','x', 'x','x','x', 'x','x','x', 'x','x','x']
maximum_shifts_type4=3
#type 5 work only weekends
matrix_nurse_availability_type5 = ['','','', '','','', '','','', '','','', '','','x', 'x','x','x', 'x','x','x']


# Create Schedule
#schedule_random = Schedule(matrix_nurses_needed=matrix_nurses_needed, is_random=False)
schedule = Schedule(matrix_nurses_needed=matrix_nurses_needed)

# Create model and nurses
model = abm_scheduling.NSP_AB_Model()
#nurses = model.generate_nurses(10, 0.5, True)
list_nurse_schedules = []
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type3)
list_nurse_schedules.append(matrix_nurse_availability_type3)
list_nurse_schedules.append(matrix_nurse_availability_type4)
list_nurse_schedules.append(matrix_nurse_availability_type4)
list_nurse_schedules.append(matrix_nurse_availability_type5)
list_nurse_schedules.append(matrix_nurse_availability_type5)
list_nurse_schedules.append(matrix_nurse_availability_type5)

nurses = model.generate_nurses_from_nurse_schedules(list_nurse_schedules)
schedule.print_schedule(schedule_name="Intial Situation")

In [ ]:
nurses[10].minimum_shifts = 2
nurses[10].maximum_shifts = 4
nurses[10].print_shift_preferences()
nurses[11].minimum_shifts = 2
nurses[11].maximum_shifts = 4
nurses[11].print_shift_preferences()

In [ ]:
model.get_total_agent_satisfaction(nurses)

In [ ]:
model.print_nurse_productivity(nurses)

In [ ]:
model.show_hypothetical_max_schedule(schedule=schedule, nurses=nurses)

In [ ]:
# Default utility function (no agent satisfaction)
start_time = datetime.today()
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.9, 
                         p_to_accept_negative_change=.001, utility_function_parameters = None)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

In [ ]:
model.print_nurse_productivity(results.nurses)

In [ ]:
# Agent satisfaction utility function
utility_function_parameters = abm_scheduling.Utility_Function_Parameters()
utility_function_parameters.utility_function = 'agent_satisfaction'
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.9, 
                         p_to_accept_negative_change=.001, utility_function_parameters = utility_function_parameters)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

In [ ]:
model.print_nurse_productivity(results.nurses)